# Car Loan vs Investment Simulator

This notebook analyzes the opportunity cost of financing a car versus paying cash and investing.

**Two Strategies Compared:**
- **Strategy A (Cash)**: Pay cash for the car upfront, then invest the equivalent monthly loan payment each month
- **Strategy B (Loan + Invest)**: Make a down payment, invest the remaining cash (lump sum OR weekly DCA), pay monthly loan payments

**Investment Modes for Strategy B:**
- **Lump Sum**: Invest entire loan amount immediately at purchase
- **Weekly DCA**: Spread investment over 52 weeks to reduce sequence-of-returns risk

**What You'll Get:**
1. Single scenario visualization for a specific start date
2. Batch backtest across ~95 years of S&P 500 history
3. Comparison of Lump Sum vs Weekly DCA strategies
4. Distribution analysis (histogram and CDF overlay)
5. Summary statistics and worst-case scenarios for both modes

Run all cells in order to see the complete analysis.

In [ ]:
# Import the car_loan_sim package
import car_loan_sim

print(f"car_loan_sim version: {car_loan_sim.__version__}")

In [ ]:
# Load S&P 500 historical data
from pathlib import Path
from car_loan_sim.data import load_sp500_series

# Load the data from the SQLite database (find it relative to notebook location)
db_path = Path(__file__).parent.parent / "sp500_daily_close.db" if "__file__" in dir() else Path("../sp500_daily_close.db")
# For Jupyter, use the parent directory
import os
notebook_dir = Path(os.getcwd())
if (notebook_dir / "sp500_daily_close.db").exists():
    db_path = notebook_dir / "sp500_daily_close.db"
elif (notebook_dir.parent / "sp500_daily_close.db").exists():
    db_path = notebook_dir.parent / "sp500_daily_close.db"

sp500 = load_sp500_series(db_path)

print(f"Loaded {len(sp500)} trading days")
print(f"Date range: {sp500.index[0].date()} to {sp500.index[-1].date()}")
print(f"\nFirst 5 days:")
print(sp500.head())
print(f"\nLast 5 days:")
print(sp500.tail())

In [ ]:
# =============================================================================
# SIMULATION PARAMETERS - Modify these values to run different scenarios
# =============================================================================

PARAMS = {
    # Car Purchase Details
    "car_price": 50000,               # Total car price in dollars
    "down_payment": 5000,             # Down payment for loan strategy (dollars)
    
    # Loan Terms
    "loan_apr": 0.06,                 # Annual percentage rate (0.06 = 6%)
    "term_months": 60,                # Loan term in months (60 = 5 years)
    
    # Single Scenario Start Date
    "start_date": "2020-01-01",       # Date of car purchase (YYYY-MM-DD)
    
    # Behavioral Assumption
    "cash_invest_monthly_payment": False,  # If True: cash buyer invests monthly payment
                                          # If False: cash buyer does not invest after purchase
    
    # Loan Investment Mode
    "loan_lump_sum_invest_mode": "lump_sum",  # "lump_sum" or "dca_weekly"
    "loan_dca_weeks": 52,                      # Weeks to spread DCA investment (default 52)
}

# =============================================================================
# Calculated Values (do not modify)
# =============================================================================
from car_loan_sim.loan import monthly_payment

loan_amount = PARAMS["car_price"] - PARAMS["down_payment"]
pmt = monthly_payment(loan_amount, PARAMS["loan_apr"], PARAMS["term_months"])

print("=" * 60)
print("SIMULATION PARAMETERS")
print("=" * 60)
print(f"\nCar Purchase:")
print(f"  Car Price:      ${PARAMS['car_price']:>12,}")
print(f"  Down Payment:   ${PARAMS['down_payment']:>12,}")
print(f"  Loan Amount:    ${loan_amount:>12,}")
print(f"\nLoan Terms:")
print(f"  APR:            {PARAMS['loan_apr']*100:>12.1f}%")
print(f"  Term:           {PARAMS['term_months']:>12} months")
print(f"  Monthly Payment:${pmt:>12,.2f}")
print(f"\nSingle Scenario:")
print(f"  Start Date:     {PARAMS['start_date']}")
print(f"\nBehavioral:")
print(f"  Cash Invests Monthly: {PARAMS['cash_invest_monthly_payment']}")
print(f"\nLoan Investment Strategy:")
print(f"  Mode:           {PARAMS['loan_lump_sum_invest_mode']}")
if PARAMS['loan_lump_sum_invest_mode'] == 'dca_weekly':
    print(f"  DCA Weeks:      {PARAMS['loan_dca_weeks']}")
print("=" * 60)

In [ ]:
# Run single scenario simulation
from car_loan_sim.simulator import simulate_one

result = simulate_one(
    start_date=PARAMS["start_date"],
    car_price=PARAMS["car_price"],
    down_payment=PARAMS["down_payment"],
    loan_apr=PARAMS["loan_apr"],
    term_months=PARAMS["term_months"],
    cash_invest_monthly_payment=PARAMS["cash_invest_monthly_payment"],
    db_path=db_path,
    loan_lump_sum_invest_mode=PARAMS["loan_lump_sum_invest_mode"],
    loan_dca_weeks=PARAMS["loan_dca_weeks"],
)

print(f"Simulation complete: {len(result)} months")
print(f"Investment mode: {PARAMS['loan_lump_sum_invest_mode']}")
print(f"\nFirst few rows:")
print(result.head())
print(f"\nLast few rows:")
print(result.tail())

In [ ]:
# Plot net worth difference over time
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(result["date"], result["diff"], linewidth=2, color="blue")
ax.axhline(y=0, color="gray", linestyle="--", alpha=0.5)

ax.set_xlabel("Date")
ax.set_ylabel("Net Worth Difference ($)")
ax.set_title("Loan+Invest vs Cash Strategy: Net Worth Difference Over Time\n(Positive = Loan strategy ahead)")

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary
final_diff = result.iloc[-1]["diff"]
print(f"\nFinal net worth difference: ${final_diff:,.2f}")
if final_diff > 0:
    print("Result: Loan + Invest strategy came out ahead")
else:
    print("Result: Cash strategy came out ahead")

## Batch Backtest: All Historical Start Dates

Run the simulation for every possible start date in the S&P 500 data where the full loan term fits.

In [ ]:
# Run batch backtest across all possible start dates
from car_loan_sim.simulator import run_backtest_all_start_dates, compute_summary_stats, compute_comparison_stats, get_worst_scenarios

print("Running batch backtest (this may take a minute)...")
backtest_results = run_backtest_all_start_dates(
    car_price=PARAMS["car_price"],
    down_payment=PARAMS["down_payment"],
    loan_apr=PARAMS["loan_apr"],
    term_months=PARAMS["term_months"],
    cash_invest_monthly_payment=PARAMS["cash_invest_monthly_payment"],
    db_path=db_path,
    loan_dca_weeks=PARAMS["loan_dca_weeks"],
)

print(f"Backtest complete: {len(backtest_results)} scenarios tested")
print(f"Date range: {backtest_results['start_date'].min().date()} to {backtest_results['start_date'].max().date()}")
print(f"\nFirst few results:")
print(backtest_results.head())
print(f"\nLast few results:")
print(backtest_results.tail())

In [ ]:
# Summary Statistics - Compare Lump Sum vs DCA
comparison = compute_comparison_stats(backtest_results)

print("=" * 70)
print("BACKTEST SUMMARY STATISTICS - LUMP SUM VS DCA COMPARISON")
print("=" * 70)
print(f"Total scenarios tested: {comparison['lump_sum']['n_scenarios']:,}")
print(f"DCA period: {PARAMS['loan_dca_weeks']} weeks (~{PARAMS['loan_dca_weeks']//4} months)")

print("\n" + "-" * 70)
print("LUMP SUM STRATEGY (invest all at purchase)")
print("-" * 70)
lump = comparison['lump_sum']
print(f"Win Rate vs Cash: {lump['win_rate']:.1%}")
print(f"\nNet Worth Difference (Loan - Cash):")
print(f"  Mean:   ${lump['mean_diff']:>12,.2f}")
print(f"  Median: ${lump['median_diff']:>12,.2f}")
print(f"  Std:    ${lump['std_diff']:>12,.2f}")
print(f"  Min:    ${lump['min_diff']:>12,.2f}")
print(f"  Max:    ${lump['max_diff']:>12,.2f}")
print(f"\nPercentiles:")
print(f"  5th:  ${lump['percentile_5']:>12,.2f}")
print(f"  25th: ${lump['percentile_25']:>12,.2f}")
print(f"  75th: ${lump['percentile_75']:>12,.2f}")
print(f"  95th: ${lump['percentile_95']:>12,.2f}")

print("\n" + "-" * 70)
print(f"WEEKLY DCA STRATEGY (invest over {PARAMS['loan_dca_weeks']} weeks)")
print("-" * 70)
dca = comparison['dca_weekly']
print(f"Win Rate vs Cash: {dca['win_rate']:.1%}")
print(f"\nNet Worth Difference (Loan - Cash):")
print(f"  Mean:   ${dca['mean_diff']:>12,.2f}")
print(f"  Median: ${dca['median_diff']:>12,.2f}")
print(f"  Std:    ${dca['std_diff']:>12,.2f}")
print(f"  Min:    ${dca['min_diff']:>12,.2f}")
print(f"  Max:    ${dca['max_diff']:>12,.2f}")
print(f"\nPercentiles:")
print(f"  5th:  ${dca['percentile_5']:>12,.2f}")
print(f"  25th: ${dca['percentile_25']:>12,.2f}")
print(f"  75th: ${dca['percentile_75']:>12,.2f}")
print(f"  95th: ${dca['percentile_95']:>12,.2f}")

print("\n" + "=" * 70)
print("HEAD-TO-HEAD COMPARISON")
print("=" * 70)
print(f"Lump Sum beats DCA: {comparison['lump_beats_dca_rate']:.1%} of scenarios")
print(f"Mean advantage (Lump - DCA): ${lump['mean_diff'] - dca['mean_diff']:,.2f}")

print("\n" + "=" * 70)
print("WORST 10 SCENARIOS - LUMP SUM")
print("=" * 70)
worst_lump = get_worst_scenarios(backtest_results, 10, "diff_loan_lump")
print(worst_lump[["start_date", "nw_cash_end", "nw_loan_lump_end", "diff_loan_lump"]].to_string(index=False))

print("\n" + "=" * 70)
print("WORST 10 SCENARIOS - WEEKLY DCA")
print("=" * 70)
worst_dca = get_worst_scenarios(backtest_results, 10, "diff_loan_dca_weekly")
print(worst_dca[["start_date", "nw_cash_end", "nw_loan_dca_weekly_end", "diff_loan_dca_weekly"]].to_string(index=False))

In [ ]:
# Histogram comparison: Lump Sum vs DCA
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Lump Sum histogram
ax1 = axes[0]
ax1.hist(backtest_results["diff_loan_lump"], bins=100, color="blue", alpha=0.7, edgecolor="black", linewidth=0.5)
ax1.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Break-even")
ax1.axvline(x=backtest_results["diff_loan_lump"].median(), color="green", linestyle="-", linewidth=2, label=f"Median: ${backtest_results['diff_loan_lump'].median():,.0f}")
ax1.set_xlabel("Net Worth Difference ($)")
ax1.set_ylabel("Frequency")
ax1.set_title("Lump Sum Strategy Distribution")
ax1.legend()
ax1.grid(True, alpha=0.3)

# DCA histogram
ax2 = axes[1]
ax2.hist(backtest_results["diff_loan_dca_weekly"], bins=100, color="orange", alpha=0.7, edgecolor="black", linewidth=0.5)
ax2.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Break-even")
ax2.axvline(x=backtest_results["diff_loan_dca_weekly"].median(), color="green", linestyle="-", linewidth=2, label=f"Median: ${backtest_results['diff_loan_dca_weekly'].median():,.0f}")
ax2.set_xlabel("Net Worth Difference ($)")
ax2.set_ylabel("Frequency")
ax2.set_title(f"Weekly DCA Strategy Distribution ({PARAMS['loan_dca_weeks']} weeks)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print comparison
print(f"\nLump Sum: {(backtest_results['diff_loan_lump'] > 0).mean():.1%} win rate")
print(f"Weekly DCA: {(backtest_results['diff_loan_dca_weekly'] > 0).mean():.1%} win rate")

In [ ]:
# CDF comparison: Lump Sum vs DCA (overlay)
import numpy as np

fig, ax = plt.subplots(figsize=(12, 6))

# Sort data for CDF
lump_sorted = np.sort(backtest_results["diff_loan_lump"])
dca_sorted = np.sort(backtest_results["diff_loan_dca_weekly"])
y = np.arange(1, len(lump_sorted) + 1) / len(lump_sorted)

# Plot CDFs
ax.plot(lump_sorted, y, linewidth=2, color="blue", label="Lump Sum")
ax.plot(dca_sorted, y, linewidth=2, color="orange", label=f"Weekly DCA ({PARAMS['loan_dca_weeks']} weeks)")

# Break-even line
ax.axvline(x=0, color="red", linestyle="--", linewidth=2, alpha=0.7, label="Break-even")

# Find loss rates
lump_loss_rate = (backtest_results["diff_loan_lump"] < 0).mean()
dca_loss_rate = (backtest_results["diff_loan_dca_weekly"] < 0).mean()
ax.axhline(y=lump_loss_rate, color="blue", linestyle=":", alpha=0.5)
ax.axhline(y=dca_loss_rate, color="orange", linestyle=":", alpha=0.5)

ax.set_xlabel("Net Worth Difference ($)", fontsize=12)
ax.set_ylabel("Cumulative Probability", fontsize=12)
ax.set_title("CDF: Lump Sum vs Weekly DCA\n(Loan+Invest Strategy vs Cash Strategy)", fontsize=14)
ax.legend(loc="lower right")
ax.grid(True, alpha=0.3)

# Add annotations
ax.annotate(f"Lump Sum loss rate: {lump_loss_rate:.1%}", 
            xy=(lump_sorted[0], lump_loss_rate), 
            xytext=(lump_sorted[int(len(lump_sorted)*0.1)], lump_loss_rate + 0.05),
            fontsize=10, color="blue")
ax.annotate(f"DCA loss rate: {dca_loss_rate:.1%}", 
            xy=(dca_sorted[0], dca_loss_rate), 
            xytext=(dca_sorted[int(len(dca_sorted)*0.1)], dca_loss_rate + 0.1),
            fontsize=10, color="orange")

plt.tight_layout()
plt.show()

## Results Interpretation

### What the Plots Show

**Histograms**: Show the distribution of outcomes for each strategy. The x-axis is the net worth difference (Loan strategy - Cash strategy). Positive values mean the loan strategy won.

**CDF (Cumulative Distribution Function)**: Shows the probability of achieving a given outcome or worse. The red vertical line at $0 shows the break-even point. The intersection with each CDF curve shows the "loss rate" for that strategy.

### Lump Sum vs Weekly DCA

**Lump Sum**: Invest the entire loan amount immediately at purchase.
- Maximizes time in market
- Higher variance (more extreme outcomes both ways)
- Better in rising markets

**Weekly DCA**: Spread investment over 52 weeks (1 year).
- Reduces sequence-of-returns risk
- Lower variance (more moderate outcomes)
- Better in volatile or falling markets

### Key Takeaways

1. **Win Rate**: The percentage of historical scenarios where each strategy came out ahead of the cash buyer. Higher is better.

2. **Lump Sum typically has higher mean**: Because markets tend to rise over time, investing earlier (lump sum) usually beats investing later (DCA).

3. **DCA has lower worst-case losses**: By spreading out investments, DCA reduces the impact of buying at a market peak.

4. **Percentiles (P5/P95 spread)**: DCA typically has a narrower spread, indicating less variability in outcomes.

### Caveats

- **Past ≠ Future**: Historical performance doesn't guarantee future results
- **Price-Only Data**: These results don't include dividends (~2%/year)
- **No Taxes**: Actual after-tax returns would differ
- **Behavioral Discipline**: Assumes cash buyer actually invests monthly (if enabled)

In [ ]:
# Save results to CSV
import os

# Create results directory if it doesn't exist
results_dir = Path("../results") if Path("../results").exists() or not Path("results").exists() else Path("results")
results_dir.mkdir(exist_ok=True)

csv_path = results_dir / "backtest_results.csv"
backtest_results.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")
print(f"File contains {len(backtest_results)} rows")
print(f"\nColumns saved: {list(backtest_results.columns)}")